In [2]:
import pygwalker as pyg
import pandas as pd
import streamlit_shadcn_ui as ui
import streamlit as st


In [3]:
# Carregando os dados
compra_itens = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/compra_itens.csv', delimiter=';')
produtos = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/produto.csv', delimiter=';', encoding='latin-1')
compras = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/compra.csv', sep=';', encoding='latin1')
fornecedores = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/fornecedores.csv', sep=';', encoding='latin1')
vendas=pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/venda.csv', delimiter=';', encoding='latin-1')
venda_itens = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/venda_itens.csv', delimiter=';', encoding='latin1')
clientes=pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/clientes.csv', delimiter=';', encoding='latin-1')

# Mesclando as tabelas vendas e clientes
venda_cliente = pd.merge(vendas[['FLOJA','IDPEDIDO','IDCLI','SI_CAIXA','FDATAEMI','HORA','FVALVENDA']],clientes[['IDFORN','NOMEFORN']],how='left',left_on='IDCLI',right_on='IDFORN')
venda_cliente.rename(columns= {'NOMEFORN': 'NOMECLI'},inplace=True)
venda_cliente.drop('IDFORN', axis='columns')

# Substituindo NaN por Consumidor Final
venda_cliente['NOMECLI'].fillna('Consumidor Final', inplace=True)

In [4]:
compra_fornecedor_dia = pd.merge(compras[['IDFORN','FDATAEMI','FVALCOMPRA']],fornecedores[['IDFORN','NOMEFORN','UF']],how='left',left_on='IDFORN',right_on='IDFORN')
resultado = compra_fornecedor_dia.groupby(['IDFORN', 'FDATAEMI', 'NOMEFORN'])['FVALCOMPRA'].sum().reset_index()
resultado

,IDFORN,FDATAEMI,NOMEFORN,FVALCOMPRA
0,2,08/04/2023,GRANFER,1291.75
1,5,08/07/2023,LDF MAT. CONSTR,1679.87
2,5,08/10/2023,LDF MAT. CONSTR,1156.06
3,5,08/28/2023,LDF MAT. CONSTR,594.86
4,11,08/02/2023,PRD,42374.22
5,16,08/10/2023,MEGA NORDESTE,1770.24
6,16,08/15/2023,MEGA NORDESTE,5328.79
7,33,08/21/2023,SEGURA EPI,242.00
8,78,08/02/2023,ALUMIFER,115.00
9,78,08/28/2023,ALUMIFER,1857.07


In [7]:
# ----------
vendas_relacao_produtos = pd.merge(produtos[['PCOD','PDESC','PVLUVEN3','PVLUVENDA']],venda_itens[['PCOD','QTDVENDA','VLVENDA']],right_on='PCOD',left_on='PCOD')
vendas_relacao_produtos = vendas_relacao_produtos.groupby(['PCOD','PDESC','PVLUVEN3','PVLUVENDA'])[['QTDVENDA','VLVENDA']].sum().reset_index()
vendas_relacao_produtos = vendas_relacao_produtos.sort_values(by='VLVENDA',ascending=False)
vendas_relacao_produtos['CUSTOCMV'] = vendas_relacao_produtos['PVLUVEN3'] * vendas_relacao_produtos['QTDVENDA']
vendas_relacao_produtos['MARGEM'] = vendas_relacao_produtos['VLVENDA'] - vendas_relacao_produtos['CUSTOCMV']
vendas_relacao_produtos['LUCRO'] = ((vendas_relacao_produtos['MARGEM']) / (vendas_relacao_produtos['VLVENDA'])) * 100
vendas_relacao_produtos['MARKUP'] = (((vendas_relacao_produtos['VLVENDA']) / (vendas_relacao_produtos['CUSTOCMV'])) * 100) -100
vendas_relacao_produtos = vendas_relacao_produtos[['PCOD','PDESC','QTDVENDA','VLVENDA','CUSTOCMV','MARGEM','LUCRO','MARKUP']]
vendas_relacao_produtos['PCOD'] = vendas_relacao_produtos['PCOD'].map(lambda x: f'{x:}')
soma_quantidade = vendas_relacao_produtos['QTDVENDA'].sum()
vendas_relacao_produtos['QTDVENDA'] = vendas_relacao_produtos['QTDVENDA'].map(lambda x: f'{x:}')

st.write(vendas_relacao_produtos)


soma_valor_venda = vendas_relacao_produtos['VLVENDA'].sum() # R$
soma_custo_cmv = vendas_relacao_produtos['CUSTOCMV'].sum() # R$
soma_margem = vendas_relacao_produtos['MARGEM'].sum() # R$
percentual_lucro_final = (soma_margem / soma_valor_venda) * 100
percentual_markup_final = ((soma_valor_venda / soma_custo_cmv) * 100) - 100

vendas_relacao_produtos

,PCOD,PDESC,QTDVENDA,VLVENDA,CUSTOCMV,MARGEM,LUCRO,MARKUP
53,4870,ESCORA METALICA 31O SEM CANECA,633.0,94488.66,55488.7800,38999.8800,41.274667,70.284263
45,4108,ESCORA METALICA 31O CM MD L,200.0,39600.00,23614.0000,15986.0000,40.368687,67.697129
14,291,GIRICA CH14 REF ROLAMENTO/ ESF,12.0,18560.32,6652.6800,11907.6400,64.156437,178.990121
61,5042,FORCADO DUPLO P/ TORRE,184.0,17451.96,10837.6000,6614.3600,37.900385,61.031594
21,375,VIGA METALICA V8 2.00 MT,66.0,11946.00,7287.0600,4658.9400,39.000000,63.934426
...,...,...,...,...,...,...,...,...
52,4806,"PORTA DE SHAFT 31,5 x 93",1.0,105.29,38.0500,67.2400,63.861715,176.714849
0,144,TUBO PATENTE 1.1/4 (42.40) CH 2.00MM,4.67,84.00,33.4372,50.5628,60.193810,151.217207
39,3465,PORTA DE SHAFT 26 X 90,1.0,82.26,33.9700,48.2900,58.704109,142.154843
22,481,CHAPA PRETA (2.65) 12 (3X120),8.0,80.00,48.4000,31.6000,39.500000,65.289256
